In [1]:
import numpy as np
import sys
sys.path.insert(0,'../')
import caffe
print caffe

<module 'caffe' from '../caffe/__init__.py'>


In [13]:
proto_str="""
input: "data"
input_dim: 1
input_dim: 1
input_dim: 3
input_dim: 4

input: "offset"
input_dim: 1
input_dim: 18
input_dim: 3
input_dim: 4

layer{
    name: "dconv"
    type: "DeformableConvolution"
    bottom: "data"
    bottom: "offset"
    top: "dconv"
    param{
        lr_mult: 1
        decay_mult: 1
    }
    param{
        lr_mult: 2
        decay_mult: 0
    }
    convolution_param{
        num_output: 1
        kernel_size: 3
        pad: 1
        stride: 1
        weight_filler{
            type: "constant"
            value: 0
        }
        bias_filler{
            type: "constant"
            value: 0
        }
    }
    loss_weight: 1
}
"""
proto_file=open('deform_conv.prototxt','w')
proto_file.write(proto_str)
proto_file.close()

In [3]:
caffe.set_device(0)
caffe.set_mode_gpu()
net = caffe.Net('deform_conv.prototxt', caffe.TEST)

In [17]:
N, C, H, W =  (1, 2, 3, 4)
np.random.seed(0)
input_data = np.random.rand(N, C, H, W)
input_offset = np.random.rand(N, 18, H, W)
input_weight = np.ones((C, C, 3, 3))
input_bias = np.zeros(C)
print input_data
print input_offset

[[[[ 0.5488135   0.71518937  0.60276338  0.54488318]
   [ 0.4236548   0.64589411  0.43758721  0.891773  ]
   [ 0.96366276  0.38344152  0.79172504  0.52889492]]

  [[ 0.56804456  0.92559664  0.07103606  0.0871293 ]
   [ 0.0202184   0.83261985  0.77815675  0.87001215]
   [ 0.97861834  0.79915856  0.46147936  0.78052918]]]]
[[[[ 0.11827443  0.63992102  0.14335329  0.94466892]
   [ 0.52184832  0.41466194  0.26455561  0.77423369]
   [ 0.45615033  0.56843395  0.0187898   0.6176355 ]]

  [[ 0.61209572  0.616934    0.94374808  0.6818203 ]
   [ 0.3595079   0.43703195  0.6976312   0.06022547]
   [ 0.66676672  0.67063787  0.21038256  0.1289263 ]]

  [[ 0.31542835  0.36371077  0.57019677  0.43860151]
   [ 0.98837384  0.10204481  0.20887676  0.16130952]
   [ 0.65310833  0.2532916   0.46631077  0.24442559]]

  [[ 0.15896958  0.11037514  0.65632959  0.13818295]
   [ 0.19658236  0.36872517  0.82099323  0.09710128]
   [ 0.83794491  0.09609841  0.97645947  0.4686512 ]]

  [[ 0.97676109  0.60484552  0.73

In [15]:
net.blobs['data'].reshape(*input_data.shape)
net.blobs['data'].data[:] = input_data.copy()
net.blobs['offset'].reshape(*input_offset.shape)
net.blobs['offset'].data[:] = input_offset.copy()
net.params['dconv'][0].reshape(*input_weight.shape)
net.params['dconv'][0].data[:] = input_weight.copy()
net.params['dconv'][1].reshape(*input_bias.shape)
net.params['dconv'][1].data[:] = input_bias.copy()
net.forward()

{'dconv': array([[[[  3.91367221,   8.10268307,   7.7055068 ,   4.8841548 ],
          [  7.0467968 ,  10.74616814,  11.21919441,   7.43512058],
          [  5.18391657,   8.17951488,   8.52711105,   5.52305508]],
 
         [[  3.91367221,   8.10268307,   7.7055068 ,   4.8841548 ],
          [  7.0467968 ,  10.74616814,  11.21919441,   7.43512058],
          [  5.18391657,   8.17951488,   8.52711105,   5.52305508]]]], dtype=float32)}

In [21]:
print net.blobs['data'].data

print net.blobs['offset'].data.shape
one_off = net.blobs['offset'].data[0,:,1,1]

[[[[ 0.54881352  0.71518934  0.60276335  0.54488319]
   [ 0.42365479  0.64589411  0.4375872   0.89177299]
   [ 0.96366274  0.38344151  0.79172504  0.5288949 ]]

  [[ 0.56804454  0.92559665  0.07103606  0.0871293 ]
   [ 0.0202184   0.83261985  0.77815676  0.87001216]
   [ 0.97861832  0.79915857  0.46147937  0.7805292 ]]]]
(1, 18, 3, 4)
[ 0.41466194  0.43703195  0.10204481  0.36872518  0.12019656  0.9292962
  0.27000797  0.29743695  0.60639322  0.65320081  0.99884701  0.45354268
  0.0185218   0.61396593  0.25435647  0.8966713   0.87265068  0.02566272]


In [ ]:
for i in range(3):
    for j in range(3):
        dx = one_off[(i*3 + j)*2 + 0]
        dy = one_off[(i*3 + j)*2 + 1]
        

In [7]:
net.backward()

{'data': array([[[[ 0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.]],
 
         [[ 0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.]]]], dtype=float32),
 'offset': array([[[[ 0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.]],
 
         [[ 0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.]],
 
         [[ 0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.]],
 
         [[ 0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.]],
 
         [[ 0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.]],
 
         [[ 0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.]],
 
         [[ 0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.]],
 
         [[ 0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.],
          [

In [8]:
print net.params['dconv'][0].diff

[[[[ 3.50684786  4.96174479  3.97533989]
   [ 5.72071314  7.45452404  5.75224209]
   [ 3.5450561   4.77297974  3.84305263]]

  [[ 3.81871986  4.54260159  3.48325324]
   [ 5.4475832   8.26427937  5.53287983]
   [ 4.20452118  5.47684002  4.16371584]]]


 [[[ 3.50684786  4.96174479  3.97533989]
   [ 5.72071314  7.45452404  5.75224209]
   [ 3.5450561   4.77297974  3.84305263]]

  [[ 3.81871986  4.54260159  3.48325324]
   [ 5.4475832   8.26427937  5.53287983]
   [ 4.20452118  5.47684002  4.16371584]]]]


In [9]:
print net.blobs['dconv'].diff
print net.blobs['input'].diff

[[[[ 1.  1.  1.  1.]
   [ 1.  1.  1.  1.]
   [ 1.  1.  1.  1.]]

  [[ 1.  1.  1.  1.]
   [ 1.  1.  1.  1.]
   [ 1.  1.  1.  1.]]]]


KeyError: 'input'

In [ ]:
print net.blobs['offset'].diff

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
x=[860,1147,1182,908,860]
y=[288,180,238,385,288]
plt.plot(x,y)
plt.axis([800,1200,400,150])